# Spotify API

## Authentication and Authorization Tokens

Load client credentials from .env file.

In [1]:
import subprocess
import sys

subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'python-dotenv'])

0

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

Use client credentials to obtain access token for Spotify API. 

See https://developer.spotify.com/documentation/web-api/tutorials/client-credentials-flow for details.

In [3]:
import requests
import time
import base64
import json
import os

TOKEN_ENDPOINT = 'https://accounts.spotify.com/api/token'
TOKEN_FILE = 'spotify_token.json'

class SpotifyAuthenticator:
    def __init__(self):
        self.access_token = None
        self.expires_at = 0
        self.load_token()
        
    def load_token(self):
        """Loads most recent access token."""
        if os.path.exists(TOKEN_FILE):
            with open(TOKEN_FILE, 'r') as file:
                data = json.load(file)
                self.access_token = data.get('access_token')
                self.expires_at = data.get('expires_at', 0)
        
    def save_token(self):
        """Saves access token for persistence."""
        with open(TOKEN_FILE, 'w') as file:
            json.dump({'access_token': self.access_token, 'expires_at': self.expires_at}, file)
        
    def get_token(self):
        """Returns access token or refreshes token if expired."""
        if not self.access_token or time.time() >= self.expires_at:
            self.refresh_token()
        return self.access_token
    
    def refresh_token(self):
        """Requests access token."""
        print('Refreshing access token...')
        
        credentials = f'{CLIENT_ID}:{CLIENT_SECRET}'
        
        # converts credentials: string -> bytes, bytes encode to Base64, bytes -> string
        encoded_credentials = base64.b64encode(credentials.encode()).decode()
        
        headers = {
            'Authorization': f'Basic {encoded_credentials}',
            'Content-Type': 'application/x-www-form-urlencoded'
        }
        data = {'grant_type': 'client_credentials'}
        
        response = requests.post(TOKEN_ENDPOINT, headers=headers, data=data)
        
        if response.status_code == 200:
            token_data = response.json()
            self.access_token = token_data['access_token']
            self.expires_at = time.time() + token_data['expires_in'] - 60
            self.save_token()
            print('Access token refreshed successfully!')
        else:
            raise Exception(f'Could not refresh access token: {response.status_code}, {response.text}')
            
spotify_authenticator = SpotifyAuthenticator()

## Querying by ISRC

In [7]:
import requests
import json

SEARCH_ENDPOINT = 'https://api.spotify.com/v1/search'

def search_isrc(isrc, category='track', limit=1):
    """Search Spotify for media using ISRC (International Standard Recording Code)."""
    token = spotify_authenticator.get_token()
    headers = {'Authorization': f'Bearer {token}'}
    
    query = f'isrc:{isrc}'
    params = {'q': query, 'type': category, 'limit': limit}

    response = requests.get(SEARCH_ENDPOINT, headers=headers, params=params)

    if response.status_code == 200:
        # return extract_track_info(response.json(), isrc)
        return response.json()
    else:
        print(f'Error {response.status_code} for ISRC {isrc}: {response.text}')
        return None

## Processing

In [5]:
import json

def extract_track_info(results, isrc):
    """Exctract track name, album name, and artist name from response."""
    items = results.get('tracks', {}).get('items', [])
    if items:
        top_result = items[0]
        return {
            'Track': top_result['name'],
            'Album Name': top_result['album']['name'],
            'Artist': top_result['artists'][0]['name']
        }
    else:
        print(f'No results found for ISRC {isrc}')
        return None

info = search_isrc('QZ7XS2400012')
if info:
    print(json.dumps(info, indent=2)) 

Refreshing access token...
Access token refreshed successfully!
{
  "Track": "CARNIVAL",
  "Album Name": "VULTURES 1",
  "Artist": "\u00a5$"
}


In [6]:
print(info)

{'Track': 'CARNIVAL', 'Album Name': 'VULTURES 1', 'Artist': '¥$'}


In [10]:
print(json.dumps(search_isrc('QZJ842400387'), indent=2))

{
  "tracks": {
    "href": "https://api.spotify.com/v1/search?offset=0&limit=1&query=isrc%3AQZJ842400387&type=track",
    "limit": 1,
    "next": "https://api.spotify.com/v1/search?offset=1&limit=1&query=isrc%3AQZJ842400387&type=track",
    "offset": 0,
    "previous": null,
    "total": 6,
    "items": [
      {
        "album": {
          "album_type": "single",
          "artists": [
            {
              "external_urls": {
                "spotify": "https://open.spotify.com/artist/0PCCGZ0wGLizHt2KZ7hhA2"
              },
              "href": "https://api.spotify.com/v1/artists/0PCCGZ0wGLizHt2KZ7hhA2",
              "id": "0PCCGZ0wGLizHt2KZ7hhA2",
              "name": "Artemas",
              "type": "artist",
              "uri": "spotify:artist:0PCCGZ0wGLizHt2KZ7hhA2"
            }
          ],
          "available_markets": [
            "AR",
            "AU",
            "AT",
            "BE",
            "BO",
            "BR",
            "BG",
            "CA",
 